In [1]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [2]:
dfAll = pd.read_pickle("PklData/AllData.pkl")
dfAll.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
0,17,1547138602677,0,34,0,0,0,False,False,"[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, ..."
1,17,1547138602697,0,34,0,0,0,False,False,"[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, ..."
2,17,1547138602796,0,34,0,0,0,False,False,"[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, ..."
3,17,1547138602817,0,34,0,0,0,False,False,"[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, ..."
4,17,1547138602863,0,34,0,0,0,False,False,"[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, ..."


In [3]:
df_actual = dfAll[(dfAll.Actual_Data == True) & (dfAll.Is_Pause == False)]
df_actual.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
7954,17,1547138928692,1,680,6,2,0,True,False,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7955,17,1547138928735,1,680,6,2,0,True,False,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7956,17,1547138928773,1,680,6,2,0,True,False,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7957,17,1547138928813,1,680,6,2,0,True,False,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7958,17,1547138928861,1,680,6,2,0,True,False,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
print("all: %s, actual data: %s" % (len(dfAll), len(df_actual)))

all: 956426, actual data: 806891


In [5]:
"""%%time
# this will be the faster repetition check
def is_max(row):
    return row.RepetitionID == df_filtered[(df_filtered.userID == row.userID) & (df_filtered.TaskID == row.TaskID) & (df_filtered.VersionID == row.VersionID)].RepetitionID.max()

pool = Pool(8)
df_filtered = df_actual.copy(deep=True)

max_vals = pool.map(is_max, [row for index, row in df_filtered.iterrows()])
df_filtered["IsMax"] = max_vals
print(df_filtered.head())
pool.close()"""

'%%time\n# this will be the faster repetition check\ndef is_max(row):\n    return row.RepetitionID == df_filtered[(df_filtered.userID == row.userID) & (df_filtered.TaskID == row.TaskID) & (df_filtered.VersionID == row.VersionID)].RepetitionID.max()\n\npool = Pool(8)\ndf_filtered = df_actual.copy(deep=True)\n\nmax_vals = pool.map(is_max, [row for index, row in df_filtered.iterrows()])\ndf_filtered["IsMax"] = max_vals\nprint(df_filtered.head())\npool.close()'

In [6]:
%%time
df_filtered = df_actual.copy(deep=True)
for User in df_filtered.userID.unique():
    print(User)
    for Task in df_filtered.TaskID.unique():
        for Version in df_filtered.VersionID.unique():
            max_rep = df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version)].RepetitionID.max()
            df_filtered = df_filtered.drop(df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version) & (df_filtered.RepetitionID != max_rep)].index)

17
2
12
15
5
1
14
10
13
6
16
3
7
8
9
11
4
CPU times: user 19min 23s, sys: 12.5 s, total: 19min 36s
Wall time: 19min 36s


In [7]:
df_filtered.to_pickle("PklData/dfFiltered_LSTM.pkl")

In [8]:
print("actual: %s, filtered data: %s" % (len(df_actual), len(df_filtered)))

actual: 806891, filtered data: 404348
